# Интерактивное обучение SQL с базой данных Chinook

## Музыкальный магазин - практические упражнения

Этот notebook поможет вам изучить SQL на примере реальной базы данных музыкального магазина.

In [ ]:
# Установка необходимых библиотек
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
# Подключение к базе данных
try:
    conn = sqlite3.connect('../data/databases/chinook.db')
    print("✅ Подключение к базе данных установлено успешно!")
except Exception as e:
    print(f"❌ Ошибка подключения: {e}")
    print("📥 Пожалуйста, загрузите базу данных с помощью скрипта download_databases.py")

In [ ]:
# Исследование структуры базы данных
print("📊 Структура базы данных Chinook:")
print("=" * 40)

# Получаем список таблиц
tables_query = """
SELECT name, sql 
FROM sqlite_master 
WHERE type = 'table'
ORDER BY name;
"""

tables_df = pd.read_sql_query(tables_query, conn)
print(f"\n📋 Найдено таблиц: {len(tables_df)}")
for idx, row in tables_df.iterrows():
    print(f"• {row['name']}")

# Показываем структуру нескольких ключевых таблиц
key_tables = ['Artist', 'Album', 'Track', 'Customer', 'Invoice']

for table in key_tables:
    print(f"\n📄 Структура таблицы {table}:")
    schema_query = f"PRAGMA table_info({table});"
    schema_df = pd.read_sql_query(schema_query, conn)
    print(schema_df[['name', 'type', 'notnull']])

## Упражнение 1: Исследование исполнителей

Найдем всех исполнителей и посмотрим на их распределение.

In [ ]:
# Показать всех исполнителей
artists_query = """
SELECT ArtistId, Name 
FROM Artist 
ORDER BY Name;
"""

artists_df = pd.read_sql_query(artists_query, conn)
print(f"📊 Всего исполнителей: {len(artists_df)}")
print("\n📋 Первые 10 исполнителей:")
print(artists_df.head(10))

# Исполнители по первой букве
first_letter_query = """
SELECT 
    SUBSTR(Name, 1, 1) as FirstLetter,
    COUNT(*) as Count
FROM Artist
GROUP BY SUBSTR(Name, 1, 1)
ORDER BY Count DESC;
"""

try:
    letter_df = pd.read_sql_query(first_letter_query, conn)
    print("\n🔤 Распределение по первой букве:")
    print(letter_df.head(10))
except:
    # Альтернативный запрос для SQLite
    letter_query = """
    SELECT 
        SUBSTR(Name, 1, 1) as FirstLetter,
        COUNT(*) as Count
    FROM Artist
    GROUP BY SUBSTR(Name, 1, 1)
    ORDER BY Count DESC
    LIMIT 10;
    """
    letter_df = pd.read_sql_query(letter_query, conn)
    print("\n🔤 Распределение по первой букве (топ-10):")
    print(letter_df)

## Упражнение 2: Анализ альбомов и треков

Посмотрим на структуру музыкальной коллекции.

In [ ]:
# Топ-10 альбомов по количеству треков
album_tracks_query = """
SELECT 
    a.Title as Album,
    ar.Name as Artist,
    COUNT(t.TrackId) as TrackCount
FROM Album a
JOIN Artist ar ON a.ArtistId = ar.ArtistId
JOIN Track t ON a.AlbumId = t.AlbumId
GROUP BY a.AlbumId
ORDER BY TrackCount DESC
LIMIT 10;
"""

album_tracks_df = pd.read_sql_query(album_tracks_query, conn)
print("🎵 Топ-10 альбомов по количеству треков:")
print(album_tracks_df)

# Визуализация
plt.figure(figsize=(12, 6))
plt.barh(album_tracks_df['Album'][:5], album_tracks_df['TrackCount'][:5])
plt.xlabel('Количество треков')
plt.title('Топ-5 альбомов по количеству треков')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Упражнение 3: Анализ продаж

Исследуем данные о продажах и клиентах.

In [ ]:
# Статистика по клиентам
customer_stats_query = """
SELECT 
    Country,
    COUNT(*) as CustomerCount,
    AVG(Total) as AvgSpent
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY Country
ORDER BY CustomerCount DESC;
"""

customer_stats_df = pd.read_sql_query(customer_stats_query, conn)
print("🌍 Статистика по странам:")
print(customer_stats_df.head(10))

# Визуализация
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Количество клиентов по странам
top_countries = customer_stats_df.head(8)
ax1.bar(top_countries['Country'], top_countries['CustomerCount'])
ax1.set_title('Количество клиентов по странам')
ax1.set_xlabel('Страна')
ax1.set_ylabel('Количество клиентов')
ax1.tick_params(axis='x', rotation=45)

# Средние траты по странам
ax2.bar(top_countries['Country'], top_countries['AvgSpent'])
ax2.set_title('Средние траты по странам')
ax2.set_xlabel('Страна')
ax2.set_ylabel('Средняя сумма ($)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Упражнение 4: Жанровой анализ

Исследуем музыкальные жанры и их популярность.

In [ ]:
# Популярность жанров
genre_query = """
SELECT 
    g.Name as Genre,
    COUNT(t.TrackId) as TrackCount,
    COUNT(DISTINCT a.AlbumId) as AlbumCount
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN Album a ON t.AlbumId = a.AlbumId
GROUP BY g.GenreId
ORDER BY TrackCount DESC;
"""

genre_df = pd.read_sql_query(genre_query, conn)
print("🎧 Популярность жанров:")
print(genre_df)

# Круговая диаграмма топ-5 жанров
top_genres = genre_df.head(5)
plt.figure(figsize=(10, 8))
plt.pie(top_genres['TrackCount'], labels=top_genres['Genre'], autopct='%1.1f%%')
plt.title('Распределение треков по топ-5 жанрам')
plt.show()

## Практические задания:

Попробуйте выполнить следующие запросы самостоятельно:

1. Найдите 5 самых дорогих треков
2. Покажите клиентов из США с их общей суммой покупок
3. Найдите исполнителей, у которых более 10 альбомов
4. Покажите месячную статистику продаж за 2021 год
5. Найдите треки, которые есть в более чем 5 плейлистах

In [ ]:
# Место для ваших решений

# Задание 1: 5 самых дорогих треков
# your_query = """
# SELECT Name, UnitPrice 
# FROM Track 
# ORDER BY UnitPrice DESC 
# LIMIT 5;
# """
# result = pd.read_sql_query(your_query, conn)
# print(result)

In [ ]:
# Закрываем соединение
conn.close()
print("✅ Соединение закрыто")